In [1]:
!pip install transformers datasets scikit-learn accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_checkpoint = "vinai/phobert-base-v2"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=3)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/540M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
import pandas as pd
from datasets import Dataset

df_train = pd.read_csv("/content/train_nons_clean2.csv")
df_val = pd.read_csv("/content/val_nons_clean2.csv")

In [5]:
aspect_order = ["stayingpower", "texture", "smell", "price", "others", "colour", "shipping", "packing"]
sentiment_map = {"negative": 0, "neutral": 1, "positive": 2}
# 8x3=24 giá trị label
#3 giá trị sẽ thuộc về 1 aspect tương ứng 3 sentiment
def encode_label(row):
    vector = [0] * (len(aspect_order) * 3)
    for i, aspect in enumerate(aspect_order):
        sentiment = row.get(aspect)
        if sentiment in sentiment_map:
            index = i * 3 + sentiment_map[sentiment]
            vector[index] = 1
    return vector

def load_and_encode(path):
    df = pd.read_csv(path)
    df["labels"] = df.apply(encode_label, axis=1)
    df["data"] = df["data"].fillna("")
    df.rename(columns={'data': 'text'}, inplace=True)
    return df[["text", "labels"]]

df_train = load_and_encode("/content/train_nons_clean2.csv")
df_val   = load_and_encode("/content/val_nons_clean2.csv")
df_test  = load_and_encode("/content/test_nons_clean2.csv")


In [ ]:
df_train

,text,labels
0,Sơn đẹp lắm sẽ ủng hộ tiếp shiper cũng dễ thươ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,Son đẹp lắm nhé mọi người tuy là son nội địa g...,"[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, ..."
2,Son màu lên chuẩn đẹp tuy nhiên dễ khô nên cần...,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,Công dụng rất đẹp kết cấu tốt độ bền màu tốt c...,"[0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,Hàng gói siêu chắc chắn packing xinh xỉu đủ te...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
12976,Nếu xinh bên phải là son mafia bên trái là her...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
12977,đóng gói cẩn thận hàng giao nhanh chưa dùng nê...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
12978,Son có mùi lạ lạ sao á lúc bôi lên môi cảm thấ...,"[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
12979,Giao hàng nhanh son đặt thứ 7 đến thứ 2 giao t...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [7]:
from datasets import Dataset
from transformers import AutoTokenizer

train_dataset = Dataset.from_pandas(df_train, preserve_index=False)
val_dataset   = Dataset.from_pandas(df_val, preserve_index=False)
test_dataset  = Dataset.from_pandas(df_test, preserve_index=False)

# Tokenize sử dụng PhoBERT tokenizer
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

train_dataset = train_dataset.map(tokenize_function)
val_dataset   = val_dataset.map(tokenize_function)
test_dataset  = test_dataset.map(tokenize_function)

# train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
# val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
# test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


Map:   0%|          | 0/12981 [00:00<?, ? examples/s]

Map:   0%|          | 0/1623 [00:00<?, ? examples/s]

Map:   0%|          | 0/1623 [00:00<?, ? examples/s]

In [33]:
import torch
import torch.nn as nn
from transformers import AutoModel

class PhoBERTForMultiLabel(nn.Module):
    def __init__(self, base_model="vinai/phobert-base-v2", num_labels=24, label_smoothing=0.1):
        super().__init__()
        self.phobert = AutoModel.from_pretrained(base_model)
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(self.phobert.config.hidden_size, num_labels)
        self.label_smoothing = label_smoothing
        self.loss_fn = nn.BCEWithLogitsLoss()

    def forward(self, input_ids, attention_mask=None, labels=None, **kwargs):
        outputs = self.phobert(input_ids=input_ids, attention_mask=attention_mask)
        pooled = outputs.last_hidden_state[:, 0, :]  # dùng [CLS] token
        pooled = self.dropout(pooled)
        logits = self.classifier(pooled)

        if labels is not None:
            # Label smoothing: kéo 0 về smoothing/2, 1 về 1 - smoothing/2
            smooth_labels = labels.float() * (1 - self.label_smoothing) + 0.5 * self.label_smoothing
            loss = self.loss_fn(logits, smooth_labels)
            return {"loss": loss, "logits": logits}
        return {"logits": logits}

In [9]:
from sklearn.metrics import (
    f1_score, accuracy_score, hamming_loss, classification_report
)
import numpy as np
aspects = ["stayingpower", "texture", "smell", "price", "others", "colour", "shipping", "packing"]
sentiments = ["NEG", "NEU", "POS"]

def split_aspect_sentiment(y_batch):
    y = np.array(y_batch).reshape(-1, 8, 3)  # (B, 8, 3)
    aspect_labels = (y.sum(axis=-1) > 0).astype(int)  # (B, 8)
    sentiment_labels = np.argmax(y, axis=-1)          # (B, 8)
    sentiment_mask = (y.sum(axis=-1) > 0)
    return aspect_labels, sentiment_labels, sentiment_mask

def compute_absa_metrics(logits, labels):
    probs = 1 / (1 + np.exp(-logits))  # sigmoid
    preds = (probs > 0.5).astype(int)

    asp_pred, sent_pred, sent_mask_pred = split_aspect_sentiment(preds)
    asp_true, sent_true, sent_mask_true = split_aspect_sentiment(labels)

    # Aspect-level metrics
    f1_aspect = f1_score(asp_true, asp_pred, average="macro")
    acc_aspect = accuracy_score(asp_true, asp_pred)

    # Sentiment-level metrics (masked)
    if sent_mask_true.sum() == 0:
        f1_sentiment = 0.0
        acc_sentiment = 0.0
    else:
        f1_sentiment = f1_score(sent_true[sent_mask_true], sent_pred[sent_mask_true], average="macro")
        acc_sentiment = accuracy_score(sent_true[sent_mask_true], sent_pred[sent_mask_true])

    # Multi-label metrics
    f1_macro_24 = f1_score(labels, preds, average="macro")
    f1_micro_24 = f1_score(labels, preds, average="micro")
    acc_macro_24 = accuracy_score(labels, preds)
    hamming = hamming_loss(labels, preds)
    exact_match_ratio = np.mean(np.all(preds == labels, axis=1))

    metrics = {
        "f1_aspect_macro": f1_aspect,
        "acc_aspect": acc_aspect,
        "f1_sentiment_macro": f1_sentiment,
        "acc_sentiment": acc_sentiment,
        "f1_macro_24": f1_macro_24,
        "f1_micro_24": f1_micro_24,
        "acc_macro_24": acc_macro_24,
        "hamming_loss": hamming,
        "subset_accuracy": exact_match_ratio,
    }

    # F1 per aspect
    for i, asp in enumerate(aspects):
        metrics[f"f1_{asp.lower()}"] = f1_score(asp_true[:, i], asp_pred[:, i])
        metrics[f"acc_{asp.lower()}"] = accuracy_score(asp_true[:, i], asp_pred[:, i])

    # F1 per sentiment (across all aspects)
    for j, sent in enumerate(sentiments):
        true_sent_flat = []
        pred_sent_flat = []
        for i in range(8):
            mask = sent_mask_true[:, i]
            if mask.sum() == 0:
                continue
            true_sent_flat.extend((sent_true[:, i][mask] == j).astype(int))
            pred_sent_flat.extend((sent_pred[:, i][mask] == j).astype(int))
        if true_sent_flat:
            metrics[f"f1_sent_{sent.lower()}"] = f1_score(true_sent_flat, pred_sent_flat)
            metrics[f"acc_sent_{sent.lower()}"] = accuracy_score(true_sent_flat, pred_sent_flat)

    return metrics

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    return compute_absa_metrics(logits, labels)


In [34]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
import transformers
from transformers import TrainerCallback
from tqdm.auto import tqdm

# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
# train_dataset.set_format("torch")
# val_dataset.set_format("torch")




training_args = TrainingArguments(
    output_dir="/absa-phobert-v2",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    warmup_ratio=0.1,
    num_train_epochs=10,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro_24",
    report_to=[],
    disable_tqdm=False

)

model = PhoBERTForMultiLabel()

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()


Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-34-1817811032.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,F1 Aspect Macro,Acc Aspect,F1 Sentiment Macro,Acc Sentiment,F1 Macro 24,F1 Micro 24,Acc Macro 24,Hamming Loss,Subset Accuracy,F1 Stayingpower,Acc Stayingpower,F1 Texture,Acc Texture,F1 Smell,Acc Smell,F1 Price,Acc Price,F1 Others,Acc Others,F1 Colour,Acc Colour,F1 Shipping,Acc Shipping,F1 Packing,Acc Packing,F1 Sent Neg,Acc Sent Neg,F1 Sent Neu,Acc Sent Neu,F1 Sent Pos,Acc Sent Pos,Runtime,Samples Per Second,Steps Per Second
1,0.340600,0.323895,0.534764,0.306839,0.555762,0.602328,0.190407,0.622468,0.299445,0.047366,0.299445,0.000000,0.837338,0.417808,0.790511,0.000000,0.813309,0.729045,0.914356,0.757370,0.934073,0.844009,0.872458,0.619883,0.799754,0.910000,0.966728,0.412801,0.612132,0.531002,0.909620,0.723484,0.682904,9.736300,166.695000,5.238000
2,0.263600,0.259782,0.809559,0.628466,0.750960,0.837316,0.314228,0.828878,0.600739,0.025801,0.600739,0.173010,0.852742,0.752318,0.884781,0.881170,0.959951,0.976959,0.990758,0.926070,0.976587,0.897491,0.911892,0.913761,0.942083,0.955696,0.982748,0.655859,0.856924,0.680000,0.931373,0.917021,0.886336,9.728500,166.829000,5.242000
3,0.245600,0.245370,0.899986,0.717190,0.801311,0.884498,0.396605,0.869310,0.678990,0.020564,0.678990,0.803456,0.943931,0.807152,0.906962,0.870849,0.956870,0.978462,0.991374,0.939450,0.979667,0.897989,0.912508,0.943800,0.961183,0.958730,0.983980,0.741591,0.903493,0.713092,0.936887,0.949249,0.928615,9.763300,166.235000,5.224000
4,0.243000,0.235235,0.929046,0.775724,0.817905,0.901654,0.436613,0.889986,0.729513,0.017740,0.729513,0.923077,0.975354,0.872162,0.934073,0.885246,0.961183,0.973806,0.989526,0.954887,0.985213,0.936464,0.943315,0.932976,0.953789,0.953748,0.982132,0.784832,0.925245,0.709497,0.936275,0.959384,0.941789,9.744300,166.558000,5.234000
5,0.230900,0.231148,0.936098,0.797289,0.814487,0.897365,0.484522,0.901538,0.753543,0.015943,0.753543,0.911937,0.972274,0.857497,0.929144,0.957555,0.984596,0.969040,0.987677,0.952030,0.983980,0.937198,0.943931,0.949780,0.964880,0.953748,0.982132,0.765743,0.914522,0.718447,0.937806,0.959272,0.942402,9.752300,166.422000,5.230000
6,0.222900,0.229293,0.947431,0.829328,0.822170,0.903493,0.497381,0.905932,0.772027,0.015429,0.772027,0.907298,0.971041,0.918297,0.956254,0.971524,0.989526,0.973643,0.989526,0.957009,0.985829,0.950171,0.955022,0.950791,0.965496,0.950715,0.980900,0.779110,0.920956,0.724518,0.938725,0.962883,0.947304,9.788700,165.803000,5.210000
7,0.220500,0.227689,0.951195,0.834874,0.834884,0.909926,0.517645,0.909346,0.779421,0.014941,0.779421,0.927203,0.976587,0.920635,0.956870,0.978369,0.991990,0.972136,0.988909,0.958801,0.986445,0.953804,0.958102,0.947741,0.963648,0.950872,0.980900,0.801070,0.931679,0.740841,0.941483,0.962741,0.946691,9.816100,165.340000,5.196000
8,0.220500,0.227705,0.953341,0.848429,0.839802,0.912990,0.524707,0.910079,0.788047,0.014864,0.788047,0.920696,0.974738,0.929864,0.961799,0.978369,0.991990,0.973725,0.989526,0.955224,0.985213,0.955932,0.959951,0.959024,0.971041,0.953895,0.982132,0.815765,0.938419,0.740346,0.940257,0.963295,0.947304,9.789600,165.789000,5.210000
9,0.215100,0.226880,0.953329,0.851510,0.843083,0.914216,0.540303,0.912330,0.789279,0.014531,0.789279,0.915385,0.972890,0.932127,0.963031,0.981758,0.993222,0.972136,0.988909,0.954717,0.985213,0.958051,0.961799,0.962771,0.973506,0.949686,0.980283,0.815287,0.937806,0.749347,0.941176,0.964615,0.949449,9.767800,166.157000,5.221000
10,0.223600,0.226740,0.954443,0.852126,0.841132,0.912071,0.538689,0.910808,0.787431,0.014787,0.787431,0.922780,0.975354,0.929705,0.961799,0.986799,0.995071,0.970543,0.988293,0.955056,0.985213,0.960162,0.963648,0.960971,0.972274,0.949527,0.980283,0.813806,0.937194,0.746770,0.939951,0.962820,0.946998,9.732700,166.757000,5.240000


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/di

TrainOutput(global_step=8120, training_loss=0.257359949473677, metrics={'train_runtime': 3190.845, 'train_samples_per_second': 40.682, 'train_steps_per_second': 2.545, 'total_flos': 0.0, 'train_loss': 0.257359949473677, 'epoch': 10.0})

In [35]:
import os
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
import transformers
from transformers import TrainerCallback
from tqdm.auto import tqdm


output_dir = "/content/drive/MyDrive/Colab Notebooks/LabNLP/cuoi ky"
os.makedirs(output_dir, exist_ok=True)

trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)


('/content/drive/MyDrive/Colab Notebooks/LabNLP/cuoi ky/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/LabNLP/cuoi ky/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/LabNLP/cuoi ky/vocab.txt',
 '/content/drive/MyDrive/Colab Notebooks/LabNLP/cuoi ky/bpe.codes',
 '/content/drive/MyDrive/Colab Notebooks/LabNLP/cuoi ky/added_tokens.json')

In [36]:
import torch
from transformers import AutoTokenizer
from safetensors.torch import load_file

# Load model và tokenizer
output_dir = "/content/drive/MyDrive/Colab Notebooks/LabNLP/cuoi ky"
tokenizer = AutoTokenizer.from_pretrained(output_dir)

state_dict = load_file("/content/drive/MyDrive/Colab Notebooks/LabNLP/cuoi ky/model.safetensors")
model = PhoBERTForMultiLabel()
model.load_state_dict(state_dict)

model.eval()

# Từ điển ánh xạ chỉ số sang aspect & sentiment
aspects = ["stayingpower", "texture", "smell", "price", "others", "colour", "shipping", "packing"]
sentiments = ['Negative', 'Neutral', 'Positive']

# Hàm hiển thị kết quả
def predict_absa(text):
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True, padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs["logits"]
        probs = torch.sigmoid(logits).cpu().numpy()[0]
        preds = (probs > 0.5).astype(int)

    aspect_sentiment = {}
    for i, (aspect, triple) in enumerate(zip(aspects, preds.reshape(8, 3))):
        if triple.sum() == 0:
            continue
        sentiment_idx = triple.argmax()
        aspect_sentiment[aspect] = sentiments[sentiment_idx]

    return aspect_sentiment


Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [37]:
text = "Son mướt chất đẹp như son đắt tiền da môi các bạn nhưng mà màu rất ưng vỏ đẹp xinh lắm "
output = predict_absa(text)

for asp, sent in output.items():
    print(f"{asp}: {sent}")


texture: Positive
colour: Positive


In [38]:
test_results = trainer.evaluate(eval_dataset=test_dataset)

for metric, value in test_results.items():
    print(f"{metric}: {value:.4f}")


eval_loss: 0.2276
eval_f1_aspect_macro: 0.9544
eval_acc_aspect: 0.8509
eval_f1_sentiment_macro: 0.8513
eval_acc_sentiment: 0.9134
eval_f1_macro_24: 0.5473
eval_f1_micro_24: 0.9097
eval_acc_macro_24: 0.7813
eval_hamming_loss: 0.0150
eval_subset_accuracy: 0.7813
eval_f1_stayingpower: 0.9248
eval_acc_stayingpower: 0.9754
eval_f1_texture: 0.9307
eval_acc_texture: 0.9587
eval_f1_smell: 0.9742
eval_acc_smell: 0.9901
eval_f1_price: 0.9726
eval_acc_price: 0.9889
eval_f1_others: 0.9625
eval_acc_others: 0.9858
eval_f1_colour: 0.9533
eval_acc_colour: 0.9587
eval_f1_shipping: 0.9690
eval_acc_shipping: 0.9797
eval_f1_packing: 0.9479
eval_acc_packing: 0.9809
eval_f1_sent_negative: 0.8138
eval_acc_sent_negative: 0.9375
eval_f1_sent_neutral: 0.7773
eval_acc_sent_neutral: 0.9415
eval_f1_sent_positive: 0.9628
eval_acc_sent_positive: 0.9479
eval_runtime: 10.0747
eval_samples_per_second: 161.0960
eval_steps_per_second: 5.0620
epoch: 10.0000


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [39]:
from termcolor import colored
import numpy as np

def print_test_predictions(model, tokenizer, test_texts, test_labels, aspect_names, sentiment_names):
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    for idx, (text, label) in enumerate(zip(test_texts, test_labels)):
        # Tokenize và predict
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = model(**inputs)
            probs = torch.sigmoid(outputs["logits"]).cpu().numpy()[0]
            preds = (probs > 0.5).astype(int)

        # So sánh
        match = (preds == label).astype(int)
        match_ratio = match.sum() / len(match)

        # Gán màu
        if match_ratio == 1.0:
            color = "green"
        elif match_ratio >= 0.5:
            color = "yellow"
        else:
            color = "red"

        # Hiển thị kết quả
        print(colored(f"{text}", color))
        print(colored("Prediction:", color), decode_prediction(preds, aspect_names, sentiment_names))
        print(colored("Ground truth:", "cyan"), decode_prediction(label, aspect_names, sentiment_names))
        print("-" * 80)

# Hàm decode vector [24] → dict {"aspect": "sentiment"}
def decode_prediction(vec, aspects, sentiments):
    vec = np.array(vec).reshape(8, 3)
    result = {}
    for i, triplet in enumerate(vec):
        if triplet.sum() == 0:
            continue
        result[aspects[i]] = sentiments[triplet.argmax()]
    return result


In [40]:
test_df = test_dataset.to_pandas()

In [41]:
aspects = ["stayingpower", "texture", "smell", "price", "others", "colour", "shipping", "packing"]
sentiments = ["NEG", "NEU", "POS"]

test_texts = test_df['text'].tolist()
test_labels = test_df['labels'].tolist()

print_test_predictions(model, tokenizer, test_texts[:10], test_labels[:10], aspects, sentiments)


Hàng đóng gói đẹp và chắc chắn nhìn rất dễ thương và ưng bụng ạ
Prediction: {'packing': 'POS'}
Ground truth: {'packing': 'POS'}
--------------------------------------------------------------------------------
Cảm giác son bên trong rất là ít luôn và đóng gói cẩn thận dịch nhưng mà giao hàng khá nhanh
Prediction: {'shipping': 'POS', 'packing': 'POS'}
Ground truth: {'shipping': 'POS', 'packing': 'POS'}
--------------------------------------------------------------------------------
Son siêu đẹp luôn ý mà ý hình mà chụp có thể không giống lắm nhưng nhìn ngoài thì giống nhé chất son mềm mướt nói chung là rất thích
Prediction: {'texture': 'POS'}
Ground truth: {'texture': 'POS'}
--------------------------------------------------------------------------------
Siêu đẹp luôn shop đóng gói cẩn thận lắm luôn lại thêm cả quà nữa nói chung là thích lắm
Prediction: {'packing': 'POS'}
Ground truth: {'packing': 'POS'}
--------------------------------------------------------------------------------
Các